In [15]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# visualisation des données

In [16]:
import plotly
print(plotly.__version__)

5.18.0


In [17]:
###############################
####     Apercu Données     ###
###############################
data = pd.read_csv('data/group3/config_1/9_2.csv')
u = data.describe().T.drop('count', axis=1).T
display(u)

,t,raw_acceleration_x,raw_acceleration_y,raw_acceleration_z,rotation_speed_x,rotation_speed_y,rotation_speed_z,magnetic_field_x,magnetic_field_y,magnetic_field_z
mean,5.918949,-0.476562,0.272031,-9.847969,-0.000469,-0.000781,-0.126719,19.562656,32.545938,19.145313
std,0.432942,2.352189,2.673165,0.258083,0.071168,0.014508,0.574374,13.041832,10.616563,14.520867
min,5.195035,-6.430000,-5.450000,-10.630000,-0.250000,-0.050000,-1.510000,-11.370000,15.060000,-23.870000
25%,5.550989,-2.142500,-1.660000,-9.940000,-0.020000,-0.002500,-0.465000,16.900000,22.560000,9.510000
50%,5.905525,-0.865000,0.145000,-9.860000,0.000000,0.000000,0.000000,24.370000,30.370000,19.180000
75%,6.293660,0.285000,2.437500,-9.730000,0.010000,0.010000,0.260000,26.250000,39.750000,29.500000
max,6.648196,4.590000,6.520000,-8.910000,0.420000,0.040000,1.210000,39.000000,50.870000,42.560000


In [18]:
#####################################
## integration discrete            ##
#####################################

def double_integrate(df, col):
    df.sort_values(by='t', inplace=True)
    velocities = df[col].cumsum()
    positions = velocities.cumsum()
    return positions

data['pos_x'] = double_integrate(data,'raw_acceleration_x')
data['pos_y'] = double_integrate(data,'raw_acceleration_y')
data['pos_z'] = double_integrate(data,'raw_acceleration_z')

In [22]:
sns.set_style("darkgrid")

# x1 = data.pos_x
# y1 = data.pos_y
# z1 = data.pos_z

x1 = data.pos_x #* (9.81*data.magnetic_field_x)
y1 = data.pos_y #* (9.81*data.magnetic_field_y)
z1 = data.pos_z #* (9.81*data.magnetic_field_z)

plt.figure(figsize=(6, 5))
fig = px.scatter_3d(data, x=x1, y=y1, z=z1)
fig.show()

<Figure size 600x500 with 0 Axes>

In [20]:
def calculate_position_irregular(acceleration, timesteps):
    position = np.zeros_like(acceleration)
    cumulative_dt = np.cumsum(timesteps)
    for i in range(1, len(acceleration)):
        dt1 = timesteps[i - 1]
        dt2 = timesteps[i]
        average_acceleration = (acceleration[i - 1] + acceleration[i]) / 2
        position[i] = position[i - 1] + dt1 * average_acceleration + dt2 * average_acceleration
    return position

# Example usage:
timesteps = np.array([0.1, 0.2, 0.3, 0.4])  # Irregular timesteps
acceleration = np.array([1, 2, 3, 4])  # Sample acceleration data

data['position_x'] = calculate_position_irregular(data.raw_acceleration_x, data.t)
data['position_y'] = calculate_position_irregular(data.raw_acceleration_y, data.t)
data['position_z'] = calculate_position_irregular(data.raw_acceleration_z, data.t)

In [21]:
plt.figure(figsize=(6, 5))
fig = px.scatter_3d(data, x=position_x, y=position_y, z=position_z)
fig.show()

<Figure size 600x500 with 0 Axes>